# DSG: ENSEMBLE

In [74]:
import numpy as np
import pandas as pd
import os

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score, train_test_split,StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from lib_DSG import ColumnSelector, DenseTransformer
folder = os.getcwd() ; print folder

/home/arda/Documents/youtube


In [75]:
pd_train = pd.read_csv('./data/train_sample_munged.csv', header=0, escapechar='\\', quotechar='"', low_memory=False)
pd_test = pd.read_csv('./data/test_sample_munged.csv', header=0, escapechar='\\', quotechar='"', low_memory=False)

In [76]:
pd_train = pd_train.fillna('')
pd_test = pd_test.fillna('')

# Simple model

In [95]:
extra = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                     min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,
                     bootstrap=False, oob_score=False, n_jobs=1, random_state=None, verbose=0, 
                     warm_start=False, class_weight=None)

bag = BaggingClassifier(base_estimator=None, n_estimators=100, max_samples=1.0, max_features=1.0, bootstrap=True,
                  bootstrap_features=False, oob_score=False, n_jobs=1, random_state=None, verbose=0)

ada = AdaBoostClassifier(base_estimator=None, n_estimators=200, learning_rate=0.3, algorithm='SAMME.R', 
                         random_state=None)

rf= RandomForestClassifier(n_estimators=10)

clf = bag

In [96]:
features = [u'viewCount', u'likeCount', u'dislikeCount',
       u'commentCount', u'duration', u'caption',
       u'licensedContent', u'dimension_2d', u'dimension_3d', u'definition_hd', u'definition_sd']

In [97]:
Y = pd_train[u'video_category_id'].values
X = pd_train[features]

In [98]:
alphas= [0]

In [99]:
results=[]
for i in alphas:
    #clf.alpha = i

    sss = StratifiedShuffleSplit(Y, 3, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X, Y,scoring='accuracy',cv=sss, n_jobs=-1)

    print ("SSS: acc: %0.4f, std: %0.4f, alpha: %s" %(scores_sss.mean(), scores_sss.std(), i))

SSS: acc: 0.3243, std: 0.0010, alpha: 0


In [ ]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }
tfv_title = TfidfVectorizer(lowercase=True, stop_words='english', token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 2), max_df=1.0, min_df=5, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

tfv_desc = TfidfVectorizer(lowercase=True, stop_words='english', token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 2), max_df=1.0, min_df=5, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

clf = MultinomialNB(alpha=0.05, fit_prior=True, class_prior=None)

title_pipe = make_pipeline(ColumnSelector(key='title'), tfv_title)
desc_pipe = make_pipeline(ColumnSelector(key='description'), tfv_desc)

pipeline = make_union(title_pipe, desc_pipe)

Index([u'video_category_id', u'title', u'description', u'published_at',
       u'viewCount', u'likeCount', u'dislikeCount', u'favoriteCount',
       u'commentCount', u'duration', u'dimension', u'definition', u'caption',
       u'licensedContent', u'topicIds', u'relevantTopicIds', u'dimension_2d',
       u'dimension_3d', u'definition_hd', u'definition_sd'],
      dtype='object')